In [3]:
%pip install avro==1.10.0 kafka-python3

Note: you may need to restart the kernel to use updated packages.


In [4]:
# import required libraries
from kafka import KafkaConsumer, KafkaProducer
import avro.schema
import avro.io
import io
import hashlib, json

ModuleNotFoundError: No module named 'kafka.vendor.six.moves'

In [ ]:
def serialize(schema, obj):
    bytes_writer = io.BytesIO()
    encoder = avro.io.BinaryEncoder(bytes_writer)
    writer = avro.io.DatumWriter(schema)
    writer.write(obj, encoder)
    return bytes_writer.getvalue()

In [ ]:
def deserialize(schema, raw_bytes):
    bytes_reader = io.BytesIO(raw_bytes)
    decoder = avro.io.BinaryDecoder(bytes_reader)
    reader = avro.io.DatumReader(schema)
    return reader.read(decoder)

In [ ]:
schema_file = 'transaction.avsc'
txschema = avro.schema.parse(open(schema_file).read())
schema_file = 'submit.avsc'
submitschema = avro.schema.parse(open(schema_file).read())
schema_file = 'result.avsc'
resultschema = avro.schema.parse(open(schema_file).read())

In [ ]:
# Connect to kafka broker running in your local host (docker). Change this to your kafka broker if needed
kafka_broker = 'lab.aimet.tech:9092'

In [ ]:
producer = KafkaProducer(bootstrap_servers=[kafka_broker])

In [ ]:
txconsumer = KafkaConsumer(
    'transaction',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(txschema, x))
resultconsumer = KafkaConsumer(
    'result',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(resultschema, x))

In [ ]:
def gen_signature(txid, payer, payee, amount, token):
    o = {'txid': txid, 'payer': payer, 'payee': payee, 'amount': amount, 'token': token}
    return hashlib.md5(json.dumps(o, sort_keys=True).encode('utf-8')).hexdigest()

In [ ]:
vid = "V595758"
token = "79e66461ffa3f8de26d3b26a41e38318"
for message in txconsumer:
    print(message.value)
    signature = gen_signature(message.value["txid"], message.value["payer"], message.value["payee"], message.value["amount"], token)
    submit = {'vid': vid, 'txid': message.value["txid"], 'signature': signature}
    producer.send('submit', serialize(submitschema, submit))
    for result in resultconsumer:
        print(result.value)
        break
    break

{'txid': 'TX07281', 'payer': 'A69985', 'payee': 'A15196', 'amount': 1854}
{'timestamp': 1711292735, 'vid': 'V675541', 'txid': 'TX07281', 'checksum': 'b679fcea2a4f946279e5c3c1ab818d00', 'code': 200, 'message': 'Confirm'}
